In [1]:
import src.config as config

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

#fetch raw data for last 28 days to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2023-06-09 09:00:00')


In [3]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (1 year)
    """

    from_date_ = from_date - timedelta(days = 7*52)
    to_date_ = to_date - timedelta(days = 7*52)
    print(f'{from_date=}, {to_date=}')

    #donwload 2 files from website
    rides = load_raw_data(year = from_date_.year, months = from_date_.month)
    rides = rides[rides['pickup_datetime'] >= from_date_]
    rides_2 = load_raw_data(year = to_date_.year, months = to_date_.month)
    rides_2 = rides_2[rides_2['pickup_datetime'] < to_date_]

    rides = pd.concat([rides, rides_2])

    #shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days = 7*52)
    rides = rides.sort_values(by = ['pickup_location_id', 'pickup_datetime'])

    return rides

In [4]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-05-12 09:00:00'), to_date=Timestamp('2023-06-09 09:00:00')
File 2022-05 was already in local storage
File 2022-06 was already in local storage


,pickup_datetime,pickup_location_id
1393800,2023-05-12 11:11:45,1
1394277,2023-05-12 11:55:33,1
1399968,2023-05-12 12:52:17,1
1399381,2023-05-12 12:52:29,1
1405113,2023-05-12 13:03:00,1


In [6]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:00<00:00, 1203.02it/s]


In [7]:
import hopsworks

#Connect to the project
project = hopsworks.login(
    project = config.HOPSWORKS_PROJECT_NAME,
    api_key_value= config.HOPSWORKS_API_KEY
)

#Connect to the feature store
feature_store = project.get_feature_store()

#connect to the feature_group
feature_group = feature_store.get_or_create_feature_group(
    name = config.FEATURE_GROUP_NAME,
    version = config.FEATURE_GROUP_VERSION,
    description = 'Time-series data at hourly frequency',
    primary_key= ['pickup_location_id', 'pickup_hour'],
    event_time = 'pickup_hour'
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/45206
Connected. Call `.close()` to terminate connection gracefully.


In [8]:
feature_group.insert(ts_data, write_options={'wait_for_job':False})

Uploading Dataframe: 0.00% |          | Rows 0/178080 | Elapsed Time: 00:00 | Remaining Time: ?

%4|1686301355.474|FAIL|Adityas-MacBook-Air.local#producer-1| [thrd:ssl://c.app.hopsworks.ai:9092/bootstrap]: ssl://c.app.hopsworks.ai:9092/0: Connection setup timed out in state SSL_HANDSHAKE (after 30062ms in state SSL_HANDSHAKE)
%5|1686301386.417|REQTMOUT|Adityas-MacBook-Air.local#producer-1| [thrd:ssl://3.142.251.253:9092/bootstrap]: ssl://3.142.251.253:9092/2: Timed out ProduceRequest in flight (after 60974ms, timeout #0)
%5|1686301386.418|REQTMOUT|Adityas-MacBook-Air.local#producer-1| [thrd:ssl://3.142.251.253:9092/bootstrap]: ssl://3.142.251.253:9092/2: Timed out ProduceRequest in flight (after 60973ms, timeout #1)
%5|1686301386.418|REQTMOUT|Adityas-MacBook-Air.local#producer-1| [thrd:ssl://3.142.251.253:9092/bootstrap]: ssl://3.142.251.253:9092/2: Timed out ProduceRequest in flight (after 60973ms, timeout #2)
%4|1686301386.419|REQTMOUT|Adityas-MacBook-Air.local#producer-1| [thrd:ssl://3.142.251.253:9092/bootstrap]: ssl://3.142.251.253:9092/2: Timed out 3 in-flight, 0 retry-queue

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/45206/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x2a332d6d0>, None)